In [1]:
# importing necessary values
!pip install sweetviz
import gzip
import shutil
import os
import pandas as pd
from ast import literal_eval
import json
from datetime import datetime
import sweetviz as sv
from sqlalchemy import create_engine

In [2]:
# reading jsons into dataframes
receipts= pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)

In [3]:
# what does the receipts dataframe look like?
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [4]:
# checking if the rewardsReceiptItemList has more than one dictionary in each row: 
# it does meaning we need separate rows for each item in the list
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'].apply(len)

0       1
1       2
2       1
3       1
4       2
       ..
1114    2
1115    2
1116    2
1117    2
1118    2
Name: rewardsReceiptItemList, Length: 1119, dtype: int64

In [5]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [6]:
# receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [7]:
receipts

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [8]:
# Get unique keys for columns containing dictionaries
def get_unique_keys(df):
    unique_keys_per_column = {col: set().union(*(d.keys() for d in df[col] if isinstance(d, dict))) for col in df.columns if df[col].apply(isinstance, args=(dict,)).any()}
    print(unique_keys_per_column)

In [9]:
get_unique_keys(receipts)

{'_id': {'$oid'}, 'createDate': {'$date'}, 'dateScanned': {'$date'}, 'finishedDate': {'$date'}, 'modifyDate': {'$date'}, 'pointsAwardedDate': {'$date'}, 'purchaseDate': {'$date'}, 'rewardsReceiptItemList': {'finalPrice', 'pointsNotAwardedReason', 'originalMetaBriteItemPrice', 'originalMetaBriteBarcode', 'needsFetchReview', 'userFlaggedBarcode', 'userFlaggedPrice', 'targetPrice', 'brandCode', 'preventTargetGapPoints', 'rewardsProductPartnerId', 'rewardsGroup', 'originalReceiptItemText', 'userFlaggedQuantity', 'competitiveProduct', 'pointsEarned', 'priceAfterCoupon', 'itemPrice', 'itemNumber', 'pointsPayerId', 'deleted', 'partnerItemId', 'userFlaggedNewItem', 'description', 'barcode', 'metabriteCampaignId', 'userFlaggedDescription', 'originalMetaBriteQuantityPurchased', 'originalMetaBriteDescription', 'originalFinalPrice', 'discountedItemPrice', 'competitorRewardsGroup', 'needsFetchReviewReason', 'quantityPurchased'}}


In [10]:
items = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')
# \
# .add_prefix('rewardsReceiptItemList.')

items.reset_index(inplace=True)

# Renaming the added column to 'RowNumber'
items.rename(columns={'index': 'RowNumber'}, inplace=True)

In [11]:
items

,RowNumber,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,7376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,7377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,7378,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,7379,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


In [12]:
del receipts['index']
receipts['RowNumber'] = receipts.index

In [13]:
receipts_final = receipts

In [14]:
def date_converter(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)

In [15]:
receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])
# create a list of the columns that need to be transformed
date_cols = ['createDate', 'dateScanned', 'finishedDate',
             'modifyDate', 'pointsAwardedDate', 'purchaseDate']

# loop over the datetime columns
for col in date_cols:
    receipts_final[col] = receipts_final[col].apply(lambda x: date_converter(x))

In [16]:
receipts_final

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,RowNumber
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,0
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,1
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,2
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,2021-01-03 00:00:00,1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,3
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41,2021-03-01 15:42:41,None,2021-03-01 15:42:41,None,NaN,None,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,7376
7377,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37,2021-03-01 14:07:37,None,2021-03-01 14:07:37,None,NaN,None,NaN,{},SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,7377
7378,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,7378
7379,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,None,2021-03-01 13:07:29,None,25.0,2020-08-17 00:00:00,2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,7379


In [17]:
receipts_final.to_csv('receipts.csv')
items.to_csv('items.csv')

In [18]:
# Create a DataFrame WITHOUT DICT objects
del receipts['rewardsReceiptItemList']

# USERS

In [19]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
...,...,...,...,...,...,...,...
490,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
491,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
492,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN
493,{'$oid': '54943462e4b07e684157a532'},True,{'$date': 1418998882381},{'$date': 1614963143204},fetch-staff,NaN,NaN


In [20]:
get_unique_keys(users)

{'_id': {'$oid'}, 'createdDate': {'$date'}, 'lastLogin': {'$date'}}


In [21]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
date_cols = ['createdDate', 'lastLogin']

# loop over the datetime columns
for col in date_cols:
    users[col] = users[col].apply(lambda x: date_converter(x))

In [22]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,NaN,NaN


In [23]:
users.to_csv('users.csv')

# BRANDS

In [24]:
brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves


In [25]:
get_unique_keys(brands)

{'_id': {'$oid'}, 'cpg': {'$id', '$ref'}}


In [26]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')

In [27]:
brands_norm

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [28]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

In [29]:
brands_final.shape

(1167, 10)

In [30]:
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])

In [31]:
brands_final

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


In [32]:
del brands_final['cpg']

In [33]:
brands_final

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


In [34]:
brands_final.to_csv('brands.csv')

# Second: Write queries that directly answer predetermined questions from a business stakeholder

Q1: What are the top 5 brands by receipts scanned for most recent month?

Q2: How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

Since both question 1 and question two are related to reach other, I am attaching both the solutions here. 

select * from
(
SELECT b.name AS Brand_Name, LEFT(purchaseDate, 7) AS split_value, COUNT(r._id) AS Receipts_Scanned
FROM Brands b
JOIN Items i ON b.barcode = i.barcode
JOIN receipts r ON i.RowNumber = r.RowNumber
-- WHERE r.dateScanned LIKE '%2021-03%'
GROUP BY b.name, split_value) as q1
ORDER BY Receipts_Scanned DESC;

| Brand_Name       | Split_Value      | Receipts_Scanned |
| ---------------- | ---------------- | -----------------|
| Tostitos         | 2021-01   | 23              |
| Swanson          | 2021-01    | 11              |
| Cracker barrel Cheese          | 2021-01    | 10              |
| Prego          | 2021-01    | 7              |
| Diet Chris Cola          | 2021-01    | 7              |
| Pepperidge Farm          | 2021-01    | 5              |



select LEFT(r.datescanned, 7) AS split_value, i.brandcode, COUNT(r._id) AS Receipts_Scanned
from receipts r
left join
items  i
on r.RowNumber = i.RowNumber
GROUP BY split_value, i.brandcode
ORDER BY split_value DESC,  COUNT(r._id) DESC


| split_value   | brandcode      | Receipts_Scanned |
| ------------- | -------------- | ----------------- |
| 2021-03 |   | 43  |
| 2021-02 |   | 508  |
| 2021-02 | MISSION  | 2  |
| 2021-02 | NULL  | 2  |
| 2021-02 | BRAND  | 2  |
| 2021-02 | VIVA  | 1  |
| 2021-01 |   | 3987  |
| 2021-01 | NULL  | 618  |
| 2021-01 | HY-VEE  | 291  |
| 2021-01 | PEPSI  | 93  |
| 2021-01 | KROGER  | 89  |


A1. The above two queries answer both, question 1 and 2. In the first query, I have commented the where clause. Not commenting it will ideally give me the right answer to the question. However since there are lots of nulls in the brandcode column, we get an empty table. We can see that in the second query, the month of march has 43 Receipts scanned but all the brandcodes are null. Thus to answer the first question, the top 5 brands by receipts scanned are Tostitos, Swanson, Cracker Barrel and Cheese, Prego and Diet Chris cola for the Month of January   

A2. To answer question 2, The second query here shows us the top 5 of each month. Since there are only null values in the month of march, we have only 1 value


Q3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Q4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

SELECT REWARDSRECEIPTSTATUS, ROUND(AVG(CAST(TOTALSPENT AS FLOAT)),2) AS AVG_TOTAL_SPENT
FROM receipts
WHERE REWARDSRECEIPTSTATUS IN ('ACCEPTED', 'REJECTED')
GROUP BY REWARDSRECEIPTSTATUS
ORDER BY COUNT(REWARDSRECEIPTSTATUS) DESC  

| REWARDSRECEIPTSTATUS | AVG_TOTAL_SPENT |
| -------------------- | --------------- |
| REJECTED           | 19.54        |


SELECT REWARDSRECEIPTSTATUS, ROUND(sum(CAST(purchasedItemCount AS FLOAT)),2) AS SUM_OF_PURCHASED_ITEMS
FROM receipts
WHERE REWARDSRECEIPTSTATUS IN ('ACCEPTED', 'REJECTED')
GROUP BY REWARDSRECEIPTSTATUS
ORDER BY COUNT(REWARDSRECEIPTSTATUS) DESC



| REWARDSRECEIPTSTATUS | SUM_OF_PURCHASED_ITEMS |
| -------------------- | --------------- |
| REJECTED             | 740         |


A3. Question 3 and 4 are on similar lines and have mentioned the queries for the same above. As we can there is no Status that mentions Accepted. I went through the json file as well and there is no entry which has rewardsreceitstatus as 'ACCEPTED' Hence The average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’ is greater for 'Rejected' being 19.54 

A4. Since it is a similar question as question three, there is not rewardsreceiptstatus as 'Accepted' Hence, total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’ is greater for 'Rejected' being 740 


Q5. Which brand has the most spend among users who were created within the past 6 months?
Q6. Which brand has the most transactions among users who were created within the past 6 months?

SET @max_date = (SELECT MAX(CREATEDDATE) FROM USERS);
SET @six_months_ago = DATE_SUB(@max_date, INTERVAL 6 MONTH);

SELECT i.brandcode, sum(totalspent_new) AS total_brand_spend
FROM (SELECT r.userid, r.rownumber, CASE 
	WHEN r.totalspent REGEXP '[a-zA-Z]' THEN 0
    ELSE CAST(r.totalspent AS DECIMAL(10,2))
    END
    AS totalspent_new FROM RECEIPTS r
LEFT JOIN users u
ON r.userid = u._id
WHERE CREATEDDATE BETWEEN @six_months_ago AND @max_date) AS nt
JOIN items i ON nt.RowNumber = i.RowNumber
GROUP BY i.brandcode
ORDER BY sum(totalspent_new) DESC;

| brandcode | total_brand_spend |
|:---------------:|:---------------:|
|  HY-VEE   |  373558.41   |
|  KROGER   |  222538.59   |
|  BEN AND JERRYS   |  135272.06   |
|  PEPSI   |  119703.34   |


SET @max_date = (SELECT MAX(CREATEDDATE) FROM USERS);
SET @six_months_ago = DATE_SUB(@max_date, INTERVAL 6 MONTH);

SELECT i.brandcode, COUNT(r._id) AS total_brand_transactions
FROM RECEIPTS r
LEFT JOIN users u
ON r.userid = u._id
JOIN items i ON r.RowNumber = i.RowNumber
WHERE CREATEDDATE BETWEEN @six_months_ago AND @max_date
GROUP BY i.brandcode
ORDER BY COUNT(r._id) DESC;


| brandcode | total_brand_transactions |
|:---------------:|:---------------:|
|  HY-VEE   |  738   |
|  PEPSI   |  118   |
|  KLEENEX   |  115   |
|  DOLE   |  89   |

A5. Question 5 and 6 are on similar lines and have mentioned the queries for the same above. I created variable to help me get the users created in the last 6 months. I joined the three table, users receipts and items. I had to do some data manupilation since the total spend column was not all numeric. it had statements within the cells. As we can see in the output, HY-VEE has the maximum total spend for the users created in the last 6 months with $373558.41

A6.This is similar to Answer 5 exceot the data manupilation since I only had to use count on the _id column of the receipts table. As we can see in the output, HY-VEE had the maximum transactions for the users created in the last 6 months with 738 transactions.


# Third: Evaluate Data Quality Issues in the Data Provided

In [35]:
def generate_report(df, report_name):
    # Generate Sweetviz report
    report = sv.analyze(df)
    # Save the report to an HTML file
    report.show_html(f'{report_name}.html')

In [36]:
generate_report(receipts_final, 'receipts_report')

                                             |          | [  0%]   00:00 -> (? left)

Report receipts_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [37]:
generate_report(items, 'items_report')

                                             |          | [  0%]   00:00 -> (? left)

Report items_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [38]:
generate_report(users, 'users_report')

                                             |          | [  0%]   00:00 -> (? left)

Report users_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [39]:
generate_report(brands_final, 'brands_report')

                                             |          | [  0%]   00:00 -> (? left)

Report brands_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


I have used sweetviz reports to check the quality of the data and to do some initial exploratory data analysis on our dataset. 

1. Initially the data that is provided does not follow the rules of normalization. To make the data useful, I performed the data manupilation above. I came up with the final 4 tables that follow the rules of normalization and used them to query the database. 

2. I created an intermediary data table by the name of Items by exploding the receipts table to expand the rewardsReceiptItemList column. This table connects the brands table to the receipts table. Having checked the report of the items table, we can see that there are a lot of null values in most of the fields. The problem arises when there are so many null values in the barcode which is the key between brands and items. Loss of data can be observed while querying for our business questions. 

3. Brandcode which is a common column between the receipts and the brand table has a lot of null values. It is not good practice to have foreign keys with more null values. This causes loss of data while answering business questions.  

4. While importing the data into the database, I had issues changing the datatypes of the columns. The reason for this was because the data is not consistent. For example, I was not able to change the purchased date field to date time since there were sentences in some entries. Data Manipulation is an issue when the data is inconsitently collected. 


# Fourth: Communicate with Stakeholders

Subject: Data Quality Issues and Recommendations for Improvement

Hello Team,

I hope this message finds you well. I've been working on analyzing our data assets, specifically the receipts, items, users, and brands tables, to optimize our data for better insights. As we delve into this process, a few data quality issues have surfaced that I'd like to bring to your attention. I would love to get some feedback and opimions on the following points. 

Normalization and Data Manipulation:
Initially, the provided data didn't adhere to normalization rules, impacting its usefulness. I performed data manipulation to create four normalized tables, ensuring better query efficiency.

Items Table:
To connect the brands and receipts tables, I created an intermediary table named "Items." However, upon reviewing the Items table, I noticed a significant number of null values, particularly in the barcode column. This poses a challenge when querying for business questions, leading to potential data loss.

Handling Missing Values:
Several columns across the data frames exhibit a high rate of missing values. While this is expected in some cases (e.g., user-flagged items), it could be problematic, especially when joining the items and brands tables. Confirming with business stakeholders on the acceptability of these missing values is crucial for completeness.

Brand Code Discrepancy:
There's an issue with brand codes missing in the brands table, affecting data completeness when querying between the receipts and brands tables. This hampers the ability to derive business insight from the data. 

Scalabilty:
I had some issues handling the size of the dataset making it time-consuming. As the dataset grows, the current strategy may become more resource-intensive and time-consuming. Considering this, there's a potential shift towards alternative data storage systems like Cloud Based Solutions, Data Warehousing or using NoSQL databases. 

I'm eager to collaborate on resolving these issues and optimizing our data for more impactful insights. Let me know a suitable time for a discussion.

Best regards,

Kairav Pandya